read all spells and create a table with all relevant information

# setup

In [1]:
import os
import re
import pandas as pd
import numpy as np

from tqdm import tqdm

# utils

In [2]:
def parse_argument(arg):
    arg = arg.strip()
    if not arg:
        return None
    if arg == 'true':
        return True
    elif arg == 'false':
        return False
    elif arg == 'nil':
        return None
    if (arg.startswith('"') and arg.endswith('"')) or (arg.startswith("'") and arg.endswith("'")):
        return arg[1:-1]
    else:
        try:
            return eval(arg)
        except NameError:
            return arg
        except:
            return arg

In [3]:
def split_arguments(s):
    args = []
    current = []
    in_quote = None
    escape = False
    for c in s:
        if escape:
            current.append(c)
            escape = False
        elif c == '\\':
            escape = True
        elif in_quote:
            if c == in_quote:
                in_quote = None
            current.append(c)
        elif c in ('"', "'"):
            in_quote = c
            current.append(c)
        elif c == ',' and not in_quote:
            args.append(''.join(current).strip())
            current = []
        else:
            current.append(c)
    if current:
        args.append(''.join(current).strip())
    return args

In [4]:
def parse_spell_file(file_path):
    spell_dict = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.split('--')[0].strip()
            if line.startswith('spell:'):
                match = re.match(r'spell:(\w+)\((.*)\)', line)
                if match:
                    method = match.group(1)
                    params_str = match.group(2)
                    args = split_arguments(params_str)
                    parsed_args = [parse_argument(arg) for arg in args if arg]
                    if parsed_args:
                        spell_dict[method] = parsed_args[0] if len(parsed_args) == 1 else parsed_args
    return spell_dict

# main

In [5]:
root_dir = '.'

In [6]:
spells = []
for root, _, files in tqdm(os.walk(root_dir)):
    for file in files:
        if file.endswith('.lua'):
            file_path = os.path.join(root, file)
            spell_dict = parse_spell_file(file_path)
            if spell_dict:
                spell_dict['path'] = file_path
                spells.append(spell_dict)

8it [00:00, 147.53it/s]


In [7]:
file, spell_dict

('sorcerer_familiar.lua',
 {'group': 'support',
  'id': 'spellId',
  'name': 'Sorcerer familiar',
  'words': 'utevo gran res ven',
  'castSound': 'SOUND_EFFECT_TYPE_SPELL_SUMMON_SORCERER_FAMILIAR',
  'level': 200,
  'mana': 3000,
  'cooldown': 0,
  'groupCooldown': 2000,
  'needLearn': False,
  'isAggressive': False,
  'vocation': ['sorcerer;true', 'master sorcerer;true'],
  'path': './familiar/sorcerer_familiar.lua'})

In [8]:
df = pd.DataFrame(spells)
len(df), len(df.columns), df.columns

(174,
 31,
 Index(['name', 'runeId', 'id', 'level', 'magicLevel', 'needTarget',
        'isAggressive', 'allowFarUse', 'charges', 'vocation', 'path', 'words',
        'group', 'castSound', 'cooldown', 'groupCooldown', 'mana', 'soul',
        'needLearn', 'isPremium', 'isSelfTarget', 'hasParams',
        'hasPlayerNameParam', 'range', 'needCasterTargetOrDirection',
        'blockWalls', 'needDirection', 'isBlockingWalls', 'allowOnSelf',
        'impactSound', 'needWeapon'],
       dtype='object'))

In [9]:
# Convert numeric columns to integers (handling NaNs)
numeric_cols = ['id', 'level', 'magicLevel', 'mana', 'soul', 'cooldown', 'groupCooldown', 'range']
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

In [10]:
# Helper function to check if a vocation is allowed
def has_vocation(voc_list, target):
    if not isinstance(voc_list, list):
        # Handle cases where only one vocation was provided as a string
        return target.lower() in str(voc_list).lower()
    return any(target.lower() in v.lower() for v in voc_list)

In [11]:
# Create a clean boolean column for Sorcerers
df['is_sorcerer'] = df['vocation'].apply(lambda x: has_vocation(x, 'sorcerer'))

In [12]:
np.unique(df['id'])

array([   0,    1,    2,    3,    6,    9,   10,   11,   13,   19,   20,
         22,   23,   24,   29,   36,   38,   39,   42,   43,   44,   45,
         48,   49,   51,   56,   57,   59,   61,   62,   75,   76,   79,
         80,   81,   82,   84,   87,   88,   89,   90,   92,   93,   95,
        105,  106,  107,  108,  109,  110,  111,  112,  113,  118,  119,
        120,  121,  122,  123,  124,  125,  126,  127,  128,  129,  131,
        132,  133,  134,  135,  138,  139,  140,  141,  142,  143,  144,
        145,  146,  147,  148,  149,  150,  151,  152,  153,  154,  155,
        156,  157,  158,  159,  160,  166,  167,  169,  170,  173,  174,
        176,  177,  178,  191,  220,  237,  238,  239,  240,  241,  242,
        243,  244,  245,  258,  260,  261,  262,  263,  264,  265,  266,
        267,  268, 1000])

In [13]:
df.loc[df['id'] == 81]

,name,runeId,id,level,magicLevel,needTarget,isAggressive,allowFarUse,charges,vocation,...,hasPlayerNameParam,range,needCasterTargetOrDirection,blockWalls,needDirection,isBlockingWalls,allowOnSelf,impactSound,needWeapon,is_sorcerer
13,Levitate,NaN,81,12,0,NaN,False,NaN,NaN,"[druid;true, elder druid;true, knight;true, el...",...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [14]:
print("--- ID CONFLICT CHECK ---")
duplicates = df[df.duplicated('id', keep=False) & (df['id'] != 0)]

if not duplicates.empty:
    print(f"⚠️ Found {len(duplicates)} spells with conflicting IDs:")
    display(duplicates[['id', 'name', 'words', 'path']].sort_values('id'))
else:
    print("✅ All spell IDs are unique.")

--- ID CONFLICT CHECK ---
⚠️ Found 6 spells with conflicting IDs:


,id,name,words,path
5,20,Find Person,exiva,./support/find_person.lua
27,20,Find Fiend,exiva moe res,./support/find_fiend.lua
47,92,Enchant Staff,exeta vis,./conjuring/enchant_staff.lua
73,92,Conjure Wand of Darkness,exevo gran mort,./conjuring/conjure_wand_of_darkness.lua
99,174,Magic Patch,exura infir,./healing/magic_patch.lua
165,174,Mud Attack,exori infir tera,./attack/mud_attack.lua


In [15]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    # Filter, Sort by Level, then by ID
    sorc_df = df[df['is_sorcerer']].sort_values(by=['level', 'id'], ascending=[True, True])

    # Display relevant columns for a quick overview
    display(sorc_df[['level', 'id', 'name', 'words', 'mana', 'group']])

,level,id,name,words,mana,group
82,1,0,Light Stone Shower Rune,adori infir mas tera,6,support
85,1,0,Lightest Missile Rune,adori infir vis,6,support
99,1,174,Magic Patch,exura infir,6,healing
137,1,177,Buzz,exori infir vis,6,attack
120,1,178,Scorch,exevo infir flam hur,8,attack
22,7,1000,Blink,exani lux,0,support
92,8,1,Light Healing,exura,20,healing
25,8,10,Light,utevo lux,20,support
5,8,20,Find Person,exiva,20,support
146,8,169,Apprentice's Strike,exori min flam,6,attack


In [ ]:
# df.to_csv('spells.csv')